# Tugas Besar 1 Machine Learning

## About
Feedforward Neural Network algorithm with rectified linear unit (ReLU) and sigmoid activation function.

### Features

### Usage

### Tech stack
* ![Python](https://img.shields.io/badge/Python-3776AB?style=for-the-badge&logo=python&logoColor=white)

## Author
| Name | NIM |
| :--- | :---: |
| Angger Ilham Amanullah | 13521001 |
| Kelvin Rayhan Alkarim | 13521005 |
| Ditra Rizqa Amadia | 13521019 |
| Bernardus Willson | 13521021 |

<hr>

> Make sure required modules are imported

In [8]:
# Modules
import json
import numpy as np
from graphviz import Digraph, Source
from IPython.display import display
import pickle
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

### Classes

In [9]:
class Driver:
    @staticmethod
    def terminate(code):
        print(f"[■] Program terminated with code {code}")
        quit(code)

    @staticmethod
    def load_model(model_file):
        try:
            model = json.load(open(model_file, "r"))
        except:
            print(f"[✖] Error loading model from '{model_file}'")
            print("")

            Driver.terminate(1)        
            
        print(f"[✔] Successfully loaded model from '{model_file}'")
        print("")

        return model

    @staticmethod
    def load_data(data_file):
        try:
            data = pd.read_csv(data_file)
        except:
            print(f"[✖] Error loading data from '{data_file}'")
            print("")

            Driver.terminate(1)        
            
        print(f"[✔] Successfully loaded data from '{data_file}'")
        print("")

        return data

In [2]:
class Layer:
    # Initialize layer
    def __init__(self, id, layer, weight, input):
        try:
            self._id = id
            self._num_of_neuron = layer["number_of_neurons"]
            self._activation_function = layer["activation_function"]
            self._weight = weight
            self._input = input       
            self._input = np.insert(self._input, 0, 1) # Append bias of 1
            self._sigma = np.zeros(self._num_of_neuron)
            self._output = np.zeros(self._num_of_neuron)
        except:
            print(f"[✖] Error initialiazing layer")
            print("")

            Driver.terminate(1)

    # Calculate sigma
    def _calculate_sigma(self, i):
        for j in range(len(self._input)):
            self._sigma[i] += self._input[j] * self._weight[j][i]

    # Calculate activation
    def _calculate_activation(self, i):
        if self._activation_function == 'relu':
            self._output[i] = max(0, self._sigma[i])
        elif self._activation_function == 'sigmoid':
            self._output[i] = 1 / (1 + np.exp(-self._sigma[i]))
        elif self._activation_function == 'linear':
            self._output[i] = self._sigma[i]
        elif self._activation_function == 'softmax':
            exp_values = np.exp(self._sigma - np.max(self._sigma))
            self._output = exp_values / np.sum(exp_values)

    # Calculate derivative
    def _calculate_derivative(self, i):
        if self._activation_function == 'relu':
            if self._sigma[i] > 0:
                return 1
            else:
                return 0
        elif self._activation_function == 'sigmoid':
            return self._output[i] * (1 - self._output[i])
        elif self._activation_function == 'linear':
            return 1
        # TODO
        elif self._activation_function == 'softmax':
            return 0

    # Calculate forward
    def _calculate_forward(self):
        print(f"[●] Building layer {self._id}")

        # For each neuron, calculate its activation
        for i in range(self._num_of_neuron):
            self._calculate_sigma(i)
            self._calculate_activation(i)

        print(f"[✔] Successfully built layer {self._id}")
        print(f"Number of neurons: {self._num_of_neuron}")
        print(f"Activation function: {self._activation_function}")
        for i in range(len(self._output)):
            print(f"h{self._id}{i}: {self._output[i]}")
        print("")
        
        return self._output

    # Calculate backward
    def _calculate_backward(self, error, learning_rate):
        print(f"[●] Backpropagating layer {self._id}")

        

In [15]:
class FFNN:
    # Initialize model
    def __init__(self, model, x, y):
        try:
            self._layers = model["case"]["model"]["layers"]
            self._weights = model["case"]["weights"]
            self._inputs = x
            self._outputs = []
            self._outputs
            self._expect = y
            self.batch_size = 3
        except:
            print(f"[✖] Error initialiazing model")
            print("")

            Driver.terminate(1)
        
        print(f"[✔] Successfully initialized model")
        print(f"Number of features: {len(self._inputs[0])}")
        print(f"Number of layers: {len(self._layers)}")
        print(f"Number of data: {len(self._inputs)}")
        print("")

    # Assert the output with the expected output
    def test(self):
        print(f"[●] Asserting output =======================================")

        passed_output = 0
        for i in range(len(self._inputs)):
            if (np.all(self._expect['output'][i] < self._outputs[i] + self._expect['max_sse']) or np.all(self._expect['output'][i] > self._outputs[i] - self._expect['max_sse'])):
                passed_output += 1

        print(f"Test status: {'FAIL' if passed_output != len(self._inputs) else 'PASS'}")
        print(f"Test result: {passed_output}/{len(self._inputs)}")
        print("")
    
    # Build the network
    def build(self):
        np.random.seed(42)
        one_hot_encoder = OneHotEncoder()
        y_encoded = one_hot_encoder.fit_transform(y.reshape(-1, 1)).toarray()

        # Loop until max epoch
        for epoch in range(5):
            print(f"[●] Epoch {epoch + 1} ========================================")

            # Shuffle data
            indices = np.arange(len(self._inputs))
            np.random.shuffle(indices)
            X_shuffled = self._inputs[indices]
            y_shuffled = y_encoded[indices]

            # Split data into batches
            for batch_start in range(0, len(self._inputs), self.batch_size):
                X_batch = X_shuffled[batch_start:batch_start + self.batch_size]
                y_batch = y_shuffled[batch_start:batch_start + self.batch_size]

                print(X_batch)

                # For each data in batch, calculate output
                for i in range(len(X_batch)):
                    _in = X_batch[i]

                    # Feed forward
                    for j in range(len(self._layers)):
                        
                        # Create layer
                        layer = Layer(j, self._layers[j], self._weights[j], _in)

                        # Calculate forward
                        _in = layer._calculate_forward()

                    self._outputs.append(_in)
                    print(f"[✔] Successfully evaluated data {i + 1}")

                # Backpropagation (TODO)
                error = y_batch[i] - _in
                for j in range(len(self._layers) - 1, -1, -1):
                    layer = Layer(j, self._layers[j], self._weights[j], _in)
                    error = layer._calculate_backward(error, 0.01)

                print(f"[✔] Successfully evaluated batch {batch_start + 1}")

In [23]:
model_file = 'models/modified_multilayer.json'
data_file = 'models/tes.csv'

In [24]:
# Input
model_json = Driver.load_model(model_file)
data_file = Driver.load_data(data_file)


# Reset index to ensure alignment after shuffling
data_file.reset_index(drop=True, inplace=True)

# Preprocess data
X = data_file.drop(columns=['Species', 'Id'])
y = data_file['Species']

# Encode categorical labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = X_train.to_numpy()
y_train = y_train

# Initialize FFNN
model = FFNN(model_json, X_train, y_train)

# Build FNN
model.build()

# Test the output
# model.test()


[✔] Successfully loaded model from 'models/modified_multilayer.json'

[✔] Successfully loaded data from 'models/tes.csv'

[✔] Successfully initialized model
Number of features: 4
Number of layers: 2
Number of data: 8

[●] Epoch 1 ========================================
[[5.1 3.5 1.4 0.2]
 [6.4 3.2 4.5 1.5]
 [6.9 3.1 4.9 1.5]]
[●] Building layer 0
[✔] Successfully built layer 0
Number of neurons: 4
Activation function: relu
h00: 2.48
h01: 8.78
h02: 2.1599999999999984
h03: 0.0

[●] Building layer 1
[✔] Successfully built layer 1
Number of neurons: 1
Activation function: softmax
h10: 1.0

[●] Backpropagating layer 1
[●] Backpropagating layer 0
[✔] Successfully evaluated data 1
[●] Building layer 0
[✔] Successfully built layer 0
Number of neurons: 4
Activation function: relu
h00: 5.330000000000001
h01: 13.989999999999998
h02: 8.309999999999999
h03: 0.0

[●] Building layer 1
[✔] Successfully built layer 1
Number of neurons: 1
Activation function: softmax
h10: 1.0

[●] Backpropagating layer

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ad693be4-3a1e-4be5-9f4f-9f4a320647d1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>